In [6]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [7]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [8]:
from czitools import metadata_tools as czimd
from czitools import read_tools
from pylibCZIrw import czi as pyczi
import os
from tqdm.contrib.itertools import product
import numpy as np
from pathlib import Path
import requests

In [9]:
# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

In [10]:
if IN_COLAB:
    filepath = os.path.join(os.getcwd(), "data/CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi")
    zarr_path = Path(filepath[:-4] + ".ome.zarr")

if not IN_COLAB:
    defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
    filepath = os.path.join(defaultdir, "CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi")
    zarr_path = defaultdir / Path(filepath[:-4] + ".ome.zarr")

# return 6D array with dimension order STZCYXA
array6d, mdata, dim_string6d,  = read_tools.read_6darray(filepath,
                                                               output_order="STZCYX")

  0%|          | 0/30 [00:00<?, ?it/s]

## ZSTD compression

* zstd0 - You can specify the parameters of it like "explicitlevel" between -131072 and 22. (e.g. "zstd0:ExplicitLevel=10")
* zstd1 - You can specify the parameters of it like "explicitlevel" between -131072 and 22 or "preprocess" - only have HiLoByteUnpack.
  * This is only done for zstd1 and for pixel types gray16 and Bgr48. (e.g. "zstd1:ExplicitLevel=10" or "zstd1:Preprocess=HiLoByteUnpack")

To read more -: https://facebook.github.io/zstd/

In [11]:
# define the compression parameters

#cmp = "zstd0:ExplicitLevel=3"
#cmp = "zstd1:ExplicitLevel=10"
#cmp = "zstd1:Preprocess=HiLoByteUnpack;ExplicitLevel=3"
cmp1 = "zstd1:Preprocess=HiLoByteUnpack;Level=Best"
cmp2 = "zstd1:Preprocess=HiLoByteUnpack;ExplicitLevel=1"

filepath_comp1 = filepath[:-4] + "_comp1.czi"
filepath_comp2 = filepath[:-4] + "_comp2.czi"

In [12]:
# write the same CZI image back to disk but this time with using ZSTD compression

# create a CZI to write to
with pyczi.create_czi(filepath_comp1, exist_ok=True) as czidoc_w:

    # iterate over all 2D planes
    for s, t, z, c in product(range(array6d.shape[0]),
                              range(array6d.shape[1]),
                              range(array6d.shape[2]),
                              range(array6d.shape[3])):

        # writing the image with compression
        czidoc_w.write(data=array6d[s, t, z, c, ...],
                       plane={"C": c, "T": t, "Z": z},
                       scene=s,
                       compression_options=cmp1,  # valid for individual planes
                       )

  0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
# write the same CZI image back to disk but this time with using ZSTD compression

# create a CZI to write to
with pyczi.create_czi(filepath_comp2, compression_options=cmp2,
                                      exist_ok=True) as czidoc_w:

    # iterate over all 2D planes
    for s, t, z, c in product(range(array6d.shape[0]),
                              range(array6d.shape[1]),
                              range(array6d.shape[2]),
                              range(array6d.shape[3])):

        # writing the image with compression
        czidoc_w.write(data=array6d[s, t, z, c, ...],
                       plane={"C": c, "T": t, "Z": z},
                       scene=s)

  0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
size = os.path.getsize(filepath)/1024.0**2
print(f"Size of the uncompressed CZI file : {np.round(size, 2)} MB ")

size_comp1 = os.path.getsize(filepath_comp1)/1024.0**2
print(f"Size of the compressed CZI file 1 : {np.round(size_comp1, 2)} MB ")

print(f"ZSTD Compression Ratio 1: {np.round(size/size_comp1, 2)}")

size_comp2 = os.path.getsize(filepath_comp2)/1024.0**2
print(f"Size of the compressed CZI file 2 : {np.round(size_comp2, 2)} MB ")

print(f"ZSTD Compression Ratio 2: {np.round(size/size_comp2, 2)}")

Size of the uncompressed CZI file : 2.88 MB 
Size of the compressed CZI file 1 : 1.28 MB 
ZSTD Compression Ratio 1: 2.25
Size of the compressed CZI file 2 : 1.32 MB 
ZSTD Compression Ratio 2: 2.19
